(C) Forum Desai, 2020. All rights reserved.

---

# Covid-19 Dashboard

The purpose of this notebook is to visualise cases against various other parameters reported by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE) for England. This gives an overview of what may be affecting the changes in number of cases e.g. number of hospital admissions, gender or number of tests being taken.

**Note**: *Click the `Refresh data` button to view the most updated statistics <br> (If more than one button is pressed at a given time, refreshing of graphs will happen consecutively. So only after the button of the current graph resets back to 'Refresh Data', can the next graph be refreshed.)*

In [1]:
# Setup: Importing modules
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from time import sleep
from datetime import date, datetime
from uk_covid19 import Cov19API
from IPython.display import Markdown

%matplotlib inline
# Changing the number changes the size of graphs
plt.rcParams['figure.dpi'] = 100

### 1) Comparing number of cases with hospital admissions for England over time:
- Cases: The numbers of people with a positive COVID19 virus test up to the published date.
- Admissions: The numbers of patients admitted to hospital with COVID19 up to the reporting date.

*Select number of cases and/or number of admissions and choose to display them in linear scale or log scale*

In [2]:
# Loading the JSON file
with open("casesvsadmissions.json", "rt") as INFILE:
    data1 = json.load(INFILE)

In [3]:
def casesvsadmissions_parse_date(datestring):
    """ Converts a date string into a pandas datetime object. """
    return pd.to_datetime(datestring, format="%Y-%m-%d")
    
def casesvsadmissions_wrangle_data(data):
    """ Wrangles the data, creates the dataframe and fills it in with data from the JSON file. """
    datalist1 = data1['data']

    # Extracting dates from data and sorts in ascending order
    dates = [dictionary['date'] for dictionary in datalist1]
    dates.sort()

    startdate = casesvsadmissions_parse_date(dates[0]) # First date
    enddate = casesvsadmissions_parse_date(dates[-1]) # Last date

    # Making date the x-axis / index
    index = pd.date_range(startdate, enddate, freq = 'D')
    # Making a timeseries of dates with columns of data
    casesvsadmissionsdf = pd.DataFrame(index = index, columns = ['cases', 'admissions'])

    # Appending data into the casesvsadmissions table
    for entry in datalist1: # Each entry is a dictionary with date, cases and hospital admissions
        date = casesvsadmissions_parse_date(entry['date'])
        for column in ['cases', 'admissions']:
            if pd.isna(casesvsadmissionsdf.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
                casesvsadmissionsdf.loc[date, column] = value

    # Filling in any remaining gaps due to missing dates
    casesvsadmissionsdf.fillna(0.0, inplace = True)
    return casesvsadmissionsdf

casesvsadmissionsdf = casesvsadmissions_wrangle_data(data1)

In [4]:
# Creating output object for printing statements from the below function
output1 = wdg.Output()
@output1.capture(clear_output = True, wait = False) # Clearing output when it is pressed again

def casesvsadmissions_access_api():
    """ Accesses the PHE API. """
    print("Downloading data from the API...")
    
    # Inserting filters and structures required for the graph
    filters = ['areaType=nation',
               'areaName=England']

    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "admissions": "newAdmissions"
    }
    
    # Creating an API object
    api = Cov19API(filters = filters, structure = structure)
    
    casesvsadmissions = api.get_json()
    print("...Done.")
    with open("casesvsadmissions.json", "wt") as OUTF:
        json.dump(casesvsadmissions, OUTF)
    return casesvsadmissions

def casesvsadmissions_refresh_graph():
    """ Redraws the graph with a different scale to visualise updated data. """
    current = casesvsadmissions_scale.value
    if current == casesvsadmissions_scale.options[0]:
        other = casesvsadmissions_scale.options[-1]
    else:
        other = casesvsadmissions_scale.options[0]
    casesvsadmissions_scale.value = other
    casesvsadmissions_scale.value = current

# Creating output object for printing statements from the below function
output2 = wdg.Output()
@output2.capture(clear_output = True, wait = False) # Clearing output when it is pressed again

def casesvsadmissions_api_button_callback(button):
    """ Button accesses API, wrangles data, updates data frame used for plotting. """
    try:
        # Updating data from API
        apidata = casesvsadmissions_access_api()
        
        # Wrangling the updated data and overwriting the dataframe
        global df
        df = casesvsadmissions_wrangle_data(apidata)

        # Refreshing the graph
        casesvsadmissions_refresh_graph()
        
        # Outputting the date and time at which the data was updated
        current_date = date.today().strftime('%dth %B %Y')
        current_time = datetime.now().strftime('%H:%M')
        print(f"Data was last refreshed at: {current_time} on {current_date}")

        # Switching the button to "Updated" if API call was successful
        casesvsadmissions_apibutton.description = "Updated"
        casesvsadmissions_apibutton.disabled = True
        casesvsadmissions_apibutton.button_style = "success"
        casesvsadmissions_apibutton.tooltip = "Data is updated"
        casesvsadmissions_apibutton.icon = "check-square-o"
        # Resetting the button to "Refresh data" so user can refresh data whenever they want
        sleep(5)
        casesvsadmissions_apibutton.description = "Refresh data"
        casesvsadmissions_apibutton.disabled = False
        casesvsadmissions_apibutton.button_style = "info"
        casesvsadmissions_apibutton.icon = "download"
    
    except:
        # Switching the button to "Failed" if API call has failed
        print("Could not access PHE API to refresh data")
        casesvsadmissions_apibutton.description = "Failed"
        casesvsadmissions_apibutton.button_style = "danger"
        casesvsadmissions_apibutton.icon = "exclamation"
        sleep(2)
        casesvsadmissions_apibutton.description = "Refresh data"
        casesvsadmissions_apibutton.disabled = False
        casesvsadmissions_apibutton.button_style = "info"
        casesvsadmissions_apibutton.icon = "download"
        
# Button
casesvsadmissions_apibutton = wdg.Button(
    description = 'Refresh data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Click to download current Public Health England data',
    icon = 'download',
    align = 'center'
)

In [5]:
casesvsadmissions_series = wdg.SelectMultiple(
    options = ['cases', 'admissions'],
    value = ['cases', 'admissions'],
    rows = 2,
    description = 'Statistics: ',
    disabled = False
)

casesvsadmissions_scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    layout= {'width': 'max-content'},
    description = 'Scale: ',
    disabled = False
)

# Changing between linear and log scales
def casesvsadmissions_graph(cols, scale):
    """ Changes between linear and log scales shown on the graph. """    
    if scale == 'linear':
        logscale = False
    else:
        logscale = True
    
    number_of_cols = len(cols)
    if number_of_cols > 0:
        casesvsadmissionsdf[list(cols)].plot(logy = logscale)
        plt.title('Number of Cases Compared to Number of Hospital Admissions')
        plt.xlabel('Time')
        plt.ylabel('Cases / Admissions')
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

In [6]:
# Calling the function for button callback
casesvsadmissions_apibutton.on_click(casesvsadmissions_api_button_callback)

# Formatting the visualisation of graph and controls
controls = wdg.VBox([casesvsadmissions_series, casesvsadmissions_scale, casesvsadmissions_apibutton, output1, output2])  
graph = wdg.interactive_output(casesvsadmissions_graph, {'cols': casesvsadmissions_series, 'scale': casesvsadmissions_scale})
everything = wdg.HBox([graph, controls])  
display(everything)

### 2) Comparing number of cases of males vs. females in England:
- Males / Females: The numbers of males / females confirmed to have COVID19.
- Cases: The numbers of people with a positive COVID19 virus test up to the published date.

*Select males, females and/or total to see how number of cases for both varied over time*

In [7]:
# Loading the JSON file
with open("agedistribution.json", "rt") as INFILE:
    data2 = json.load(INFILE)

In [8]:
def agedistribution_wrangle_data(data):
    """ Wrangles the data, creates the dataframe and fills it in with data from the JSON file. """
    datadict = data2['data'][0] # First day
    males = datadict['males'] # Extracting all male data from the first day
    females = datadict['females'] # Extracting all female data from the first day
    age_groups = [x['age'] for x in males] # Obtaining all age groups of males (same as females)
    
    def min_age(age_groups):
        """ Removes punctuation from age groups. """
        age_groups = age_groups.replace('+','') # remove the + from 90+
        start = age_groups.split('_')[0]
        return int(start) 
    
    # Sorting ages in ascending order
    age_groups.sort(key = min_age)
    
    agedistributiondf = pd.DataFrame(index = age_groups, columns = ['males', 'females', 'total'])
    
    # Allocating each age group with number of people  
    for entry in males:
        agegroup = entry['age']
        agedistributiondf.loc[agegroup, 'males'] = entry['value']
    
    for entry in females:
        agegroup = entry['age']
        agedistributiondf.loc[agegroup, 'females'] = entry['value']
    
    agedistributiondf['total'] = agedistributiondf['males'] + agedistributiondf['females']

    return agedistributiondf
   
agedistributiondf = agedistribution_wrangle_data(data2)

In [26]:
# Creating output object for printing statements from the below function
output3 = wdg.Output()
@output3.capture(clear_output = True, wait = False) # Clearing output when it is pressed again

def agedistribution_access_api():
    """ Accesses the PHE API. """
    print("Downloading data from the API...")
    
    # Inserting filters and structures required for the graph
    filters = ['areaType=nation',
               'areaName=England']

    structure = {
        "males": "maleCases",
        "females": "femaleCases",
    }
    
    # Creating an API object
    api = Cov19API(filters = filters, structure = structure)
    
    agedistribution = api.get_json()
    print("...Done.")
    with open("agedistribution.json", "wt") as OUTF:
        json.dump(agedistribution, OUTF)
    return agedistribution

def agedistribution_refresh_graph():
    """ Redraws the graph with a different scale to visualise updated data. """
    current = agecols.value
    agecols.value = ['males', 'females', 'total']
    sleep(1)
    agecols.value = current
        
# Creating output object for printing statements from the below function
output4 = wdg.Output()
@output4.capture(clear_output = True, wait = False) # Clearing output when it is pressed again

def agedistribution_api_button_callback(button):
    """ Button accesses API, wrangles data, updates data frame used for plotting. """    
    try:
        # Updating data from API
        apidata = agedistribution_access_api()
        
        # Wrangling the updated data and overwriting the dataframe
        global df
        df = agedistribution_wrangle_data(apidata)
        
        # Refreshing the graph
        agedistribution_refresh_graph()

        # Outputting the date and time at which the data was updated
        current_date = date.today().strftime('%dth %B %Y')
        current_time = datetime.now().strftime('%H:%M')
        print(f"Data was last refreshed at: {current_time} on {current_date}")

        # Switching the button to "Updated" if API call was successful
        agedistribution_apibutton.description = "Updated"
        agedistribution_apibutton.disabled = True
        agedistribution_apibutton.button_style = "success"
        agedistribution_apibutton.tooltip = "Data is updated"
        agedistribution_apibutton.icon = "check-square-o"
        # Resetting the button to "Refresh data" so user can refresh data whenever they want
        sleep(5)
        agedistribution_apibutton.description = "Refresh data"
        agedistribution_apibutton.disabled = False
        agedistribution_apibutton.button_style = "info"
        agedistribution_apibutton.icon = "download"
    
    except:
        # Switching the button to "Failed" if API call has failed
        print("Could not access PHE API to refresh data")
        agedistribution_apibutton.description = "Failed"     
        agedistribution_apibutton.disabled= False
        agedistribution_apibutton.button_style = "danger"
        agedistribution_apibutton.icon = "exclamation"
        sleep(2)
        agedistribution_apibutton.description = "Refresh data"
        agedistribution_apibutton.disabled = False
        agedistribution_apibutton.button_style = "info"
        agedistribution_apibutton.icon = "download"

        
# Button
agedistribution_apibutton = wdg.Button(
    description = 'Refresh data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Click to download current Public Health England data',
    icon = 'download'
)

In [27]:
agecols = wdg.SelectMultiple(
    options = ['males', 'females', 'total'],
    value = ['total'],
    rows = 3,
    description = 'Statistics: ',
    disabled = False
)

def age_graph(agecols):
    """ Formats the bar chart. """    
    number_of_cols = len(agecols)
    if number_of_cols > 0:
        agedistributiondf[list(agecols)].plot(kind = 'bar')
        plt.title('Number of Males Cases Compared to Female Cases')
        plt.xlabel('Age Group')
        plt.ylabel('Cases')
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

In [28]:
# Calling the function for button callback
agedistribution_apibutton.on_click(agedistribution_api_button_callback)

# Formatting the visualisation of graph and controls
controls = wdg.VBox([agecols, agedistribution_apibutton, output3, output4])  
graph = wdg.interactive_output(age_graph, {'agecols': agecols})
everything = wdg.HBox([graph, controls])  
display(everything)

### 3) Comparing number of cases with tests taken in England over time:
- Tests: Number of pillar 1, 2 and 3 tests taken per day (pillar 4 tests were not available for England). 
    - Data is only available from 1 April 2020. 
    - 1st April 2020 - 13th July 2020: Only the number of pillar 1 lab-based tests were reported
    - From 1st June 2020 onwards: Pillar 3 antibody tests were included
    - From 14th July 2020 onwards: Pillar 2 lab-based tests were included
- Cases: The numbers of people with a positive COVID19 virus test up to the published date.

*Select one or more of the options and choose to display them in linear scale or log scale*

In [12]:
# Loading the JSON file
with open("casesvstests.json", "rt") as INFILE:
    data3 = json.load(INFILE)

In [13]:
def casesvstests_parse_date(datestring):
    """ Converts a date string into a pandas datetime object. """
    return pd.to_datetime(datestring, format="%Y-%m-%d")
    
def casesvstests_wrangle_data(data):
    """ Wrangles the data, creates the dataframe and fills it in with data from the JSON file. """
    datalist3 = data3['data']

    # Extracting dates from data and sorts in ascending order
    dates = [dictionary['date'] for dictionary in datalist3]
    dates.sort()

    startdate = casesvstests_parse_date(dates[0]) # First date
    enddate = casesvstests_parse_date(dates[-1]) # Last date

    # Making date the x-axis / index
    index = pd.date_range(startdate, enddate, freq = 'D')
    # Making a timeseries of dates with columns of data
    casesvstestsdf = pd.DataFrame(index = index, columns = ['cases', 'pillarone', 'pillartwo', 'pillarthree', 'total'])

    # Appending data into the casesvstests table
    for entry in datalist3: # Each entry is a dictionary with date, cases and the different tests
        date = casesvstests_parse_date(entry['date'])
        for column in ['cases', 'pillarone', 'pillartwo', 'pillarthree']:
            if pd.isna(casesvstestsdf.loc[date, column]):
                value = float(entry[column]) if entry[column] != None else 0.0
                casesvstestsdf.loc[date, column] = value
    casesvstestsdf['total'] = casesvstestsdf['pillarone'] + casesvstestsdf['pillartwo'] + casesvstestsdf['pillarthree']
    
    # Filling in any remaining gaps due to missing dates
    casesvstestsdf.fillna(0.0, inplace = True)
    return casesvstestsdf

casesvstestsdf = casesvstests_wrangle_data(data3)

In [14]:
# Creating output object for printing statements from the below function
output5 = wdg.Output()
@output5.capture(clear_output = True, wait = False) # Clearing output when it is pressed again

def casesvstests_access_api():
    """ Accesses the PHE API. """
    print("Downloading data from the API...")
    
    # Inserting filters and structures required for the graph
    filters = ['areaType=nation',
               'areaName=England']

    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "pillarone": "newPillarOneTestsByPublishDate",
        "pillartwo": "newPillarTwoTestsByPublishDate",
        "pillarthree": "newPillarThreeTestsByPublishDate",
    }
    
    # Creating an API object
    api = Cov19API(filters = filters, structure = structure)
    
    casesvstests = api.get_json()
    print("...Done.")
    with open("casesvstests.json", "wt") as OUTF:
        json.dump(casesvstests, OUTF)
    return casesvstests

def casesvstests_refresh_graph():
    """ Redraws the graph with a different scale to visualise updated data. """
    current = scale.value
    if current == scale.options[0]:
        other = scale.options[-1]
    else:
        other = scale.options[0]
    scale.value = other
    scale.value = current

# Creating output object for printing statements from the below function
output6 = wdg.Output()
@output6.capture(clear_output = True, wait = False) # Clearing output when it is pressed again

def casesvstests_api_button_callback(button):
    """ Button accesses API, wrangles data, updates data frame used for plotting. """ 
    try:
        # Updating data from API
        apidata = casesvstests_access_api()
        
        # Wrangling the updated data and overwriting the dataframe
        global df
        df = casesvstests_wrangle_data(apidata)
        
        # Refreshing the graph
        casesvstests_refresh_graph()

        # Outputting the date and time at which the data was updated
        current_date = date.today().strftime('%dth %B %Y')
        current_time = datetime.now().strftime('%H:%M')
        print(f"Data was last refreshed at: {current_time} on {current_date}")

        # Switching the button to "Updated" if API call was successful
        casesvstests_apibutton.description = "Updated"
        casesvstests_apibutton.disabled = True
        casesvstests_apibutton.button_style = "success"
        casesvstests_apibutton.tooltip = "Data is updated"
        casesvstests_apibutton.icon = "check-square-o"
        # Resetting the button to "Refresh data" so user can refresh data whenever they want
        sleep(5)
        casesvstests_apibutton.description = "Refresh data"
        casesvstests_apibutton.disabled = False
        casesvstests_apibutton.button_style = "info"
        casesvstests_apibutton.icon = "download"
    
    except:
        # Switching the button to "Failed" if API call has failed
        print("Could not access PHE API to refresh data")
        casesvstests_apibutton.description = "Failed"
        casesvstests_apibutton.button_style = "danger"
        casesvstests_apibutton.icon = "exclamation"
        sleep(2)
        casesvstests_apibutton.description = "Refresh data"
        casesvstests_apibutton.disabled = False
        casesvstests_apibutton.button_style = "info"
        casesvstests_apibutton.icon = "download"
        
# Button
casesvstests_apibutton = wdg.Button(
    description = 'Refresh data',
    disabled = False,
    button_style = 'info',
    tooltip = 'Click to download current Public Health England data',
    icon = 'download'
)

In [15]:
series = wdg.SelectMultiple(
    options = ['cases', 'pillarone', 'pillartwo', 'pillarthree', 'total'],
    value = ['cases','total'],
    rows = 5,
    description = 'Statistics: ',
    disabled = False
)

scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    layout={'width': 'max-content'}, # If the items' names are long
    description = 'Scale: ',
    disabled = False
)

# Changing between linear and log scales
def tests_graph(cols, scale):
    """ Changes between linear and log scales shown on the graph. """    
    if scale == 'linear':
        logscale = False
    else:
        logscale = True
    
    number_of_cols = len(cols)
    if number_of_cols > 0:
        casesvstestsdf[list(cols)].plot(logy = logscale)
        plt.title('Number of Cases Compared to Number of Tests Taken')
        plt.xlabel('Time')
        plt.ylabel('Cases / Tests')
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

In [16]:
# Calling the function for button callback
casesvstests_apibutton.on_click(casesvstests_api_button_callback)

# Formatting the visualisation of graph and controls
controls = wdg.VBox([series, scale, casesvstests_apibutton, output5, output6])  
graph = wdg.interactive_output(tests_graph, {'cols': series, 'scale': scale})
everything = wdg.HBox([graph, controls])  
display(everything)

---
**Data source**: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*